# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# import numpy:
import numpy as np
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import numpy.random as rnd
from IPython.display import display
from textwrap import wrap
# Using CV2
import cv2
from sklearn.utils import shuffle
# Load pickled data
import pickle
import csv


In [ ]:
# TODO: Fill this in based on where you saved the training and testing data

training_file = "train.p"
validation_file = "valid.p"
testing_file = "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
with open('signnames.csv', mode='r') as mapping_file:
    reader = csv.reader(mapping_file)
    signs = {rows[0]:rows[1] for rows in reader}   #row0 - id, row1 - name

    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

print()
print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_valid)))
print("Test Set:       {} samples".format(len(X_test)))

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results


# TODO: Number of training examples
n_train = format(len(X_train))

# TODO: Number of testing examples.
n_test = format(len(X_test))

# TODO: What's the shape of an traffic sign image?
image_shape = format(X_train[0].shape)

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))         
#len(set(y_train)) - http://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.


# Visualizations will be shown in the notebook.
%matplotlib inline


cnum_image = 5
rnum_image = 5

for r in range(rnum_image):
    fig = plt.figure()    # next row of images
    index = rnd.randint(0, len(X_train), size = 5)
    for i in range(cnum_image):
        #The subplot() command specifies numrows, numcols, fignum where fignum ranges from 1 to numrows*numcols. 
        fig.add_subplot(1,cnum_image,i+1)
        #fig.subplots_adjust(wspace=1)
        plt.tight_layout()
        plt.imshow(X_train[index] [i])   #[i] is index no (0 to 4) , i^th image from index
        #plt.title(y_train[index] [i])
        plt.title('\n'.join(wrap(signs[str(y_train[index] [i])], 14)),fontsize=8)
        plt.axis('off')
    plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
print("Image Shape: {}".format(X_train.shape))

### Brightness Normalization  # u1/uint8 - https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html
Xb_train = np.zeros(X_train.shape,dtype='u1')
Xb_valid = np.zeros(X_valid.shape,dtype='u1')
Xb_test = np.zeros(X_test.shape,dtype='u1')

def brightness_normalization(X_in, Xb_out):
    for i in range(0, len(X_in)):  
        cv2.normalize(X_in[i],Xb_out[i],0, 255, cv2.NORM_MINMAX)
    
    return Xb_out

#res = np.hstack((X_train[255],Xb_train[255],X_train[1500],Xb_train[1500])) #stacking images side-by-side
#plt.imshow(res)

In [ ]:
Xb_train = brightness_normalization(X_train, Xb_train)
res = np.hstack((X_train[255],Xb_train[255],X_train[16000],Xb_train[16000])) #stacking images side-by-side
plt.imshow(res)

In [ ]:
def send_sms(msg):
    # we import the Twilio client from the dependency we just installed
    from twilio.rest import TwilioRestClient
    
    # the following line needs your Twilio Account SID and Auth Token
    client = TwilioRestClient("AC30533cc04155731bca1e5a3eff7c16b4", "1c6dfb9adc0e9190ac8cae2a13c6414c")
    
    # change the "from_" number to your Twilio number and the "to" number
    # to the phone number you signed up for Twilio with, or upgrade your
    # account to send SMS to any phone number
    client.messages.create(to="+18322712158", from_="+15629121012", 
                           body=msg)

In [ ]:
def pre_process_image(Xpp_in, Xpp_out):
    cv2.normalize(Xpp_in, Xpp_in, 0, 255, cv2.NORM_MINMAX)
    
    return Xpp_in

In [ ]:

def transform_image(image,ang_range,shear_range,trans_range):
# Rotation
    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = image.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)
# Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
# Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])
    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2
    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])
    shear_M = cv2.getAffineTransform(pts1,pts2)
        
    image = cv2.warpAffine(image,Rot_M,(cols,rows))
    image = cv2.warpAffine(image,Trans_M,(cols,rows))
    image = cv2.warpAffine(image,shear_M,(cols,rows))
    
    #Brightness augmentation
    #image = augment_brightness_camera_images(image)
    
    image2 = np.zeros(X_train[0].shape, dtype='u1')
    # Preprocessing 
    image2 = pre_process_image(image,image2)     # brightness Normalization
    
    return image

In [ ]:

def brightness_Augmentation(Xbtin_train, ybtin_train, Xbto_train, ybto_train):
    for i in range(0, len(Xbtin_train)):   #loop running for all train images
        if (i%5000 == 0):
            msg = "pre-processing: " + format(i) + " images processed" 
            send_sms(msg)
        for j in range(1):                #Loop produces 2 Augmented and normalized images for each base image
            image_transformed = transform_image(Xbtin_train[i],20,10,5).reshape((-1,32,32,3))    #Transformation function
            Xbto_train = np.concatenate((Xbto_train, image_transformed), axis=0)   #concatenate = Original + 2 Augmented images
            ybto_train = np.concatenate((ybto_train,ybtin_train[i].reshape((-1,)))) #label concatenation
    print(Xbtin_train.shape, Xbto_train.shape, ybto_train.shape, ybtin_train[5].shape) # final shape of dataset #can be commented
    return Xbto_train, ybto_train

#NOtes:
#reshaping and concatenation [see above, reshape((-1,32,32,3)) and reshape((-1,))]: 
#http://stackoverflow.com/documentation/numpy/1296/the-basics/6739/reshaping-and-concatenation-of-an-array#t=201702211832191612933

# Create Arrays to perform Augmentation on:
Xbtin_train = np.copy(X_train)
ybtin_train = np.copy(y_train)
Xbto_train = np.copy(Xbtin_train)
ybto_train = np.copy(ybtin_train)
print(Xbtin_train.shape, Xbto_train.shape, ybto_train.shape, ybtin_train[5].shape)

In [ ]:

start = timer()
Xbto_train, ybto_train = brightness_Augmentation(Xbtin_train, ybtin_train, Xbto_train, ybto_train)
stop = timer()
print("Time elapsed in preprocessing: ",format((stop-start)/60))

#--------------------------------------------------------------------------------------------------
msg = "pre-processing Done !!! Time taken to pre=process: " + format((stop-start)/60)
send_sms(msg)
#--------------------------------------------------------------------------------------------------

In [ ]:
print(Xbtin_train.shape, Xbto_train.shape)
res = np.hstack((Xbtin_train[255],Xbto_train[255],Xbtin_train[16000],Xbto_train[16000])) #stacking images side-by-side
res1 = np.hstack((Xb_train[1],Xbto_train[34799]))
res2 = np.hstack((Xb_train[250],Xbto_train[35099]))
res3 = np.hstack((Xb_train[1000],Xbto_train[35799]))

plt.imshow(res)
plt.imshow(np.vstack((res1,res2,res3)))
print(ybtin_train[1], ybto_train[34799])
print(ybtin_train[300], ybto_train[35099])
print(ybtin_train[1000], ybto_train[35799])

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
#commit 2740e1fd445c2d2676d420b5b24839091d7f6890


In [ ]:
#setup tensorflow
import tensorflow as tf

EPOCHS = 20
BATCH_SIZE = 128
keep_prob = tf.placeholder(tf.float32) # probability to keep units

In [ ]:
#Implement LeNet
# DropOut : https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/d5cf4454-1324-4524-9e2c-0ecca1f5c40e
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    
    # TODO: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
        #(out_height = ceil(float(in_height - filter_height + 1) / float(strides[1])))
    wc1 = tf.Variable(tf.truncated_normal((5, 5, 3, 6), mean = mu, stddev = sigma))
         # wc1 (height, width, input_depth, output_depth ,calculate filter h and w)
    wb1 = tf.Variable(tf.zeros(6))
    strides1 = [1, 1, 1, 1] # (batch, height, width, depth)
    padding1 = 'VALID'
    conv1 = tf.nn.conv2d(x, wc1, strides1, padding1) + wb1

    # TODO: Activation.
    conv1 = tf.nn.relu(conv1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    #new_height = (input_height - filter_height)/S + 1;, (put filter_height=S and solve for S)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')


    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    wc2 = tf.Variable(tf.truncated_normal((5, 5, 6, 16), mean = mu, stddev = sigma))
         # wc1 (height, width, input_depth, output_depth ,calculate filter h and w)
    wb2 = tf.Variable(tf.zeros(16))
    strides2 = [1, 1, 1, 1] # (batch, height, width, depth)
    padding2 = 'VALID'
    conv2 = tf.nn.conv2d(conv1, wc2, strides2, padding2) + wb2
    
    # TODO: Activation.
    conv2 = tf.nn.relu(conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    

    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # TODO: Activation. & #DropOut
    fc1    = tf.nn.relu(fc1)
    fc1    = tf.nn.dropout(fc1, keep_prob)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation. & #DropOut
    fc2    = tf.nn.relu(fc2)
    fc2    = tf.nn.dropout(fc2, keep_prob)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b

    
    return logits

In [ ]:
#`x` is a placeholder for a batch of input images.
#`y` is a placeholder for a batch of output labels.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the test set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

In [ ]:
#training pipeline

rate = 0.0009

logits = LeNet(x)
#cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
#Model evaluation

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:

#Xb_train = brightness_normalization(X_train, Xb_train)  #first do pre-processing
#Xb_valid = brightness_normalization(X_valid, Xb_valid)  #first do pre-processing
#Xbto_train, ybto_train = shuffle(Xbto_train, ybto_train)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(Xbto_train)
    
    start = timer()
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        Xbto_train, ybto_train = shuffle(Xbto_train, ybto_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = Xbto_train[offset:end], ybto_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.8})
        #print(end)
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    stop = timer()
    print("Time elapsed for this run: {:.2f}".format((stop-start)/60))
    
    #msg = "Model Trained and Validation accuracy is: " + format(validation_accuracy)
    #send_sms(msg)
    
    saver.save(sess, './lenet')
    print("Model saved")

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the IPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 